In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda,Compose
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# dowload Data
training_data = datasets.FashionMNIST(
    root="data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [ ]:
test_dataloader

In [ ]:
for x,y in test_dataloader:
  print(x,y)

In [ ]:
device = "cuda" if torch.cuda.is_available() else  "cpu"
device

In [ ]:
# define our custome Model

class NeuralNetworks(nn.Module):
  def __init__(self):
    super(NeuralNetworks,self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x=self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
device

In [ ]:
model = NeuralNetworks().to(device)
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)


In [ ]:
## train loop

def train(DataLoader,model,loss_fn,optimizer):
  size = len(DataLoader.dataset)
  model.train()
  for batch,(X,y) in enumerate(DataLoader):
    X,y = X.to(device),y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred,y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss,current = loss.item(),batch*len(X)
      print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(DataLoader,model,loss_fn):
  size = len(DataLoader.dataset)
  num_batches = len(DataLoader)
  model.eval()
  test_loss,correct = 0,0
  with torch.no_grad():
    for batch,(X,y) in enumerate(DataLoader):
      X,y = X.to(device),y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [ ]:
## Start training

epochs = 20
for t in range(epochs):
  print(f"Epoch {t+1}\n -----------------------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)

print("Done")


In [ ]:
## Save model
torch.save(model.state_dict(),"model.pth")
print("Model saved")

In [ ]:
model = NeuralNetworks()
model.load_state_dict(torch.load("/content/model.pth"))

In [ ]:
## Prediction classes

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

In [ ]:
len(classes)

In [ ]:
model.eval()

In [ ]:
x,y= test_data[0][0],test_data[0][1]
x,y

In [ ]:
y

In [ ]:
with torch.no_grad():
  pred = model(x)
  predicted,actual = classes[pred[0].argmax(0)],classes[y]
  print(f'Predicted :"{predicted}",Actual: "{actual}"')

In [ ]:
import torchvision
# Loop through the training dataloader to get a batch

for images,labels in train_dataloader:
  break # Get the first batch and exit loop
# Convert the images to numpy arrays
images = images.numpy()
# Plot the images
fig,axes = plt.subplots(nrows=4,ncols=4,figsize=(10,10))
for i,ax in enumerate(axes.flat):
  # Display the image
  ax.imshow(np.transpose(images[i],(1,2,0)))
  ax.set_title(f"Label:{labels[i].item()}")
  ax.axis('off')

plt.tight_layout()
plt.show()
